<a href="https://colab.research.google.com/github/brianadit24/SentimentAnalysiswithBERT_HF/blob/main/Sentiment_Analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install and Import Dependencies

In [ ]:
!pip3 install torch torchvision torchaudio

     |████████████████████████████████| 1.9MB 7.8MB/s 


In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.3MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
     |████████████████████████████████| 901kB 45.3MB/s 


In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# 3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode("I hated mas Yoza, he absolutely the worst mentor", return_tensors='pt')

In [ ]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10321, 13195, 10601,   117, 10191, 35925,
        10563, 10103, 43060, 43760,   102])

In [ ]:
tokenizer.decode(tokens[0])

'[CLS] i hated mas yoza, he absolutely the worst mentor [SEP]'

In [ ]:
# Predict Tokens
result = model(tokens)

In [ ]:
result

SequenceClassifierOutput([('logits',
                           tensor([[ 3.8262,  2.0046, -0.0161, -2.4628, -2.4295]],
                                  grad_fn=<AddmmBackward>))])

In [ ]:
predict_sentiment = int(torch.argmax(result.logits))+1
predict_sentiment

1

# 4. Collect Reviews

In [ ]:
r = requests.get('https://yelp.com/biz/8milepi-detroit-style-pizza-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

["In the past year and a half, I've fallen in love with Detroit-style pizza and been on the hunt to try every place SF, which is what lead me to 8 MilePi! 8 MilePI offers both Detroit-style and Sicilian pizza, but of course I had to go with the Detroit. I love bbq chicken pizzas, so that one immediately got my attention and placed my order for that. The bbq sauce was on the bitter side, but just wish it was a little sweeter and tangy because the bitterness of the sauce overpowered the rest of ingredients and was the prominent flavor. The crust fell short of expectations as well. It wasn't the crispy, cheesy edges that i've had with previous Detroit-style pizzas. The crust as a whole had more of the sicilian texture rather than the Detroit. \xa0The pizza itself is huge! The regular size was $28, and I was full off 2 1/2 slices. One regular sized pizza would be good for 2-4 people. The packaging can use some improvement. I ordered a regular size za' but it came delivered in their larger 

# 5. Load Reviews into DataFrame and Score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df

,review
0,"In the past year and a half, I've fallen in lo..."
1,Not sure what's up with all these haters. I gr...
2,3.5 starsOrdered via door dash and some of the...
3,"Super messy presentation, undercooked, and ver..."
4,Warning this is a ghost kitchen and not an act...
5,This pizza was straight up nasty.I ordered a D...
6,The ultimate Detroit style pizzas with that de...
7,"Hi Sonam, thank you for sharing this great fee..."
8,I got a veggie pie and it wasn't great to be h...
9,Disappointing. Not enough sauce or cheese and ...


In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df.review.iloc[0])

3

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512])) #Limited to 512 Token

In [ ]:
df

,review,sentiment
0,"In the past year and a half, I've fallen in lo...",4
1,Not sure what's up with all these haters. I gr...,5
2,3.5 starsOrdered via door dash and some of the...,3
3,"Super messy presentation, undercooked, and ver...",2
4,Warning this is a ghost kitchen and not an act...,1
5,This pizza was straight up nasty.I ordered a D...,2
6,The ultimate Detroit style pizzas with that de...,5
7,"Hi Sonam, thank you for sharing this great fee...",5
8,I got a veggie pie and it wasn't great to be h...,2
9,Disappointing. Not enough sauce or cheese and ...,2
